<a href="https://colab.research.google.com/github/ikm4rkov/OCR_examples/blob/main/passport_recognition_alpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Определение документа на изображении, удаление 3D-деформаций и бинаризация изображения осуществляются с помощью document-scanner https://github.com/endalk200/document-scanner.
Установка через pypi не удалась, поэтому был использован source. Так как он на python, не потребовалась компиляция, только зависимости.

In [ ]:
!git clone https://github.com/endalk200/document-scanner.git

Cloning into 'document-scanner'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 81 (delta 24), reused 62 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (81/81), 20.61 MiB | 13.00 MiB/s, done.
Resolving deltas: 100% (24/24), done.


С версиями в requirements.txt зависимости не установились, поэтому жесткие версии были удалены.

In [ ]:
!cat document-scanner/requirements.txt

matplotlib
numpy
ocrd-fork-pylsd
opencv-python
scipy

In [ ]:
!pip install -r document-scanner/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.7 MB/s eta 0:00:00


Document-scanner можно запустить или на одном изображении (--image), или на множестве (--images). Вне зависисмости от опции, выходные файлы называются так же как входные и сохраняются в директории output, создаваемой в директории запуска.

In [ ]:
!mkdir prescan

In [ ]:
!python document-scanner/scan.py --images prescan/

Proccessed 1.png
Proccessed 6.png
Proccessed 4.png
Proccessed 2.png
Proccessed 8.png
Proccessed 9.jpg
Proccessed 3.png
Proccessed 10.jpg
Proccessed 7.png
Proccessed 5.png


Изображения в разном порядке, имеются дубли страниц и неодинаковые ориентации.

In [ ]:
!pip install opencv-python

Ориентирование документа параллельно сторонам изображения и попытка учест по отличить повороты, кратные 90 градусам с помощью opencv. В связи с этим установка opencv-python.

In [ ]:
!cat orientation.py

import cv2
import numpy as np
import sys
import os

# Считается, что если меньше 5 градусов несоответствия ориентации документа, он выровнен
def is_aligned(angle):
    return abs(angle) < 5 
if __name__ == "__main__":
    image_path = sys.argv[1]
    image = cv2.imread(image_path)

# Проверка правильности пути
    if image is None:
        print(f"Ошибка: изображение по пути '{image_path}' не найдено.")
    else:
      # Ориентирование
        _, thresh = cv2.threshold(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), 150, 255, cv2.THRESH_BINARY_INV)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        largest_contour = max(contours, key=cv2.contourArea)
        rect = cv2.minAreaRect(largest_contour)
        angle = rect[2]

        if not is_aligned(angle):
            if angle < -45:
                angle += 90
            elif angle > 45:
                angle -= 90

            if angle > 0:
                angle = 90 - angle

            (h, w

In [49]:
!for file in output/*; do python orientation.py $file; done

Повернутое изображение сохранено как 'output/10_rotated.png'.
Повернутое изображение сохранено как 'output/1_rotated.png'.
Повернутое изображение сохранено как 'output/2_rotated.png'.
Изображение уже выровнено. Сохранено как 'output/3_rotated.png'.
Повернутое изображение сохранено как 'output/4_rotated.png'.
Изображение уже выровнено. Сохранено как 'output/5_rotated.png'.
Повернутое изображение сохранено как 'output/6_rotated.png'.
Изображение уже выровнено. Сохранено как 'output/7_rotated.png'.
Повернутое изображение сохранено как 'output/8_rotated.png'.
Изображение уже выровнено. Сохранено как 'output/9_rotated.png'.
Ошибка: изображение по пути 'output/deepdocetion_results' не найдено.
Ошибка: изображение по пути 'output/rotated' не найдено.


Распознавание структуры документа с помощью deepdoctection https://github.com/deepdoctection/deepdoctection. Работает совместно с моделью распознавания письменной речи (OCR), и предлагаются на выбор 4 модели. Из них только Tesseract и DocTR поддерживают русский, Tesseract будет использован далее.

In [ ]:
!pip install deepdoctection[tf]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 34.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of onnx to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of onnx to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Установка Tesseract

In [ ]:
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (3,541 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [ ]:
!mkdir -p output/rotated && for file in output/*_rotated*; do [[ -f "$file" ]] && base_name=$(basename "$file" | sed 's/\.[^.]*$//') && mkdir -p "output/rotated/$base_name" && cp "$file" "output/rotated/$base_name/"; done

In [ ]:
!pip install poppler-utils

In [ ]:
!wget https://github.com/tesseract-ocr/tessdata/raw/refs/heads/main/rus.traineddata -P /usr/share/tesseract-ocr/4.00/tessdata/

--2024-11-04 14:22:50--  https://github.com/tesseract-ocr/tessdata/raw/refs/heads/main/rus.traineddata
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tesseract-ocr/tessdata/refs/heads/main/rus.traineddata [following]
--2024-11-04 14:22:50--  https://raw.githubusercontent.com/tesseract-ocr/tessdata/refs/heads/main/rus.traineddata
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19920885 (19M) [application/octet-stream]
Saving to: ‘/usr/share/tesseract-ocr/4.00/tessdata/rus.traineddata’

rus.traineddata     100%[===================>]  19.00M  --.-KB/s    in 0.1s    

2024-11-04 14:22:51 (140 MB/s)

In [36]:
!for dir in output/rotated/*/; do python deepdoctection.py "$dir"; done

Traceback (most recent call last):
  File "/content/deepdoctection.py", line 12, in <module>
    analyzer = dd.get_dd_analyzer(config_overwrite=["LANGUAGE='rus'"])
AttributeError: module 'deepdoctection' has no attribute 'get_dd_analyzer'
Traceback (most recent call last):
  File "/content/deepdoctection.py", line 12, in <module>
    analyzer = dd.get_dd_analyzer(config_overwrite=["LANGUAGE='rus'"])
AttributeError: module 'deepdoctection' has no attribute 'get_dd_analyzer'
Traceback (most recent call last):
  File "/content/deepdoctection.py", line 12, in <module>
    analyzer = dd.get_dd_analyzer(config_overwrite=["LANGUAGE='rus'"])
AttributeError: module 'deepdoctection' has no attribute 'get_dd_analyzer'
Traceback (most recent call last):
  File "/content/deepdoctection.py", line 12, in <module>
    analyzer = dd.get_dd_analyzer(config_overwrite=["LANGUAGE='rus'"])
AttributeError: module 'deepdoctection' has no attribute 'get_dd_analyzer'
Traceback (most recent call last):
  File "/

In [46]:
!mkdir output/deepdocetion_results

In [48]:
import deepdoctection as dd
import os
from pathlib import Path


analyzer = dd.get_dd_analyzer(config_overwrite=["LANGUAGE='rus'"])

path = Path("output/rotated")
subdirs = [d for d in path.iterdir() if d.is_dir()]

for i in subdirs:
  df = analyzer.analyze(path=i)
  df.reset_state()

  doc = iter(df)
  page = next(doc)

  base_name = os.path.splitext(page.file_name)[0]
  print(base_name)
  # Сохранение таблицы в файл
  if page.tables:
      tables_file_path = f"output/deepdocetion_results/{base_name}_tables.html"
      with open(tables_file_path, 'w') as tables_file:
          tables_file.write(page.tables[0].html)
      print(f"Таблицы сохранены в '{tables_file_path}'")
  else:
      print("Таблицы не найдены.")

  # Сохранение текста в файл
  try:
      text_content = '\n'.join(page.text_)
  except ValueError:
      text_content = ''

  text_file_path = f"output/deepdocetion_results/{base_name}_text.txt"
  with open(text_file_path, 'w') as text_file:
      text_file.write(text_content)
  print(f"Текст сохранен в '{text_file_path}'")

[1104 15:28.06 @dd.py:472]  INF  Config: 
 {'DEVICE': device(type='cpu'),
 'LANGUAGE': 'rus',
 'LAYOUT_NMS_PAIRS': {'COMBINATIONS': None, 'PRIORITY': None, 'THRESHOLDS': None},
 'LIB': 'PT',
 'OCR': {'CONFIG': {'TESSERACT': 'dd/conf_tesseract.yaml'},
         'USE_DOCTR': False,
         'USE_TESSERACT': True,
         'USE_TEXTRACT': False,
         'WEIGHTS': {'DOCTR_RECOGNITION': {'PT': 'doctr/crnn_vgg16_bn/pt/crnn_vgg16_bn-9762b0b0.pt',
                                           'TF': 'doctr/crnn_vgg16_bn/tf/crnn_vgg16_bn-76b7f2c6.zip'},
                     'DOCTR_WORD': {'PT': 'doctr/db_resnet50/pt/db_resnet50-ac60cadc.pt',
                                    'TF': 'doctr/db_resnet50/tf/db_resnet50-adcafc63.zip'}}},
 'PDF_MINER': {'X_TOLERANCE': 3, 'Y_TOLERANCE': 3},
 'PT': {'CELL': {'FILTER': None,
                 'WEIGHTS': 'cell/d2_model_1849999_cell_inf_only.pt',
                 'WEIGHTS_TS': 'cell/d2_model_1849999_cell_inf_only.ts'},
        'ITEM': {'FILTER': None,
      

[PosixPath('output/rotated/4_rotated'), PosixPath('output/rotated/5_rotated'), PosixPath('output/rotated/8_rotated'), PosixPath('output/rotated/6_rotated'), PosixPath('output/rotated/3_rotated'), PosixPath('output/rotated/10_rotated'), PosixPath('output/rotated/9_rotated'), PosixPath('output/rotated/7_rotated'), PosixPath('output/rotated/1_rotated'), PosixPath('output/rotated/2_rotated')]
output/rotated/4_rotated


|          |1/?[00:00<00:00,1956.30it/s]
[1104 15:28.11 @doctectionpipe.py:84]  INF  Processing 4_rotated.png
[1104 15:28.25 @context.py:133]  INF  ImageLayoutService total: 13.9091 sec.
[1104 15:28.52 @context.py:133]  INF  SubImageLayoutService total: 27.5935 sec.
[1104 15:29.16 @context.py:133]  INF  SubImageLayoutService total: 23.3888 sec.
[1104 15:29.16 @context.py:133]  INF  TableSegmentationService total: 0.5221 sec.
[1104 15:29.17 @context.py:133]  INF  TableSegmentationRefinementService total: 0.1722 sec.
[1104 15:29.18 @context.py:133]  INF  TextExtractionService total: 1.5377 sec.
[1104 15:29.18 @context.py:133]  INF  MatchingService total: 0.0087 sec.
[1104 15:29.18 @context.py:133]  INF  TextOrderService total: 0.1005 sec.
[1104 15:29.22 @view.py:326]  WRN  html construction not possible
[1104 15:29.22 @view.py:326]  WRN  html construction not possible
[1104 15:29.22 @view.py:326]  WRN  html construction not possible
[1104 15:29.22 @view.py:326]  WRN  html construction no

4_rotated


[1104 15:29.22 @view.py:326]  WRN  html construction not possible
[1104 15:29.22 @view.py:326]  WRN  html construction not possible
[1104 15:29.22 @view.py:326]  WRN  html construction not possible
[1104 15:29.22 @view.py:326]  WRN  html construction not possible
[1104 15:29.22 @view.py:326]  WRN  html construction not possible
[1104 15:29.22 @view.py:326]  WRN  html construction not possible
[1104 15:29.22 @view.py:326]  WRN  html construction not possible
[1104 15:29.22 @view.py:326]  WRN  html construction not possible
[1104 15:29.22 @view.py:326]  WRN  html construction not possible
[1104 15:29.22 @view.py:326]  WRN  html construction not possible
[1104 15:29.22 @view.py:326]  WRN  html construction not possible
[1104 15:29.22 @view.py:326]  WRN  html construction not possible
[1104 15:29.22 @view.py:326]  WRN  html construction not possible
[1104 15:29.22 @view.py:326]  WRN  html construction not possible
[1104 15:29.22 @view.py:326]  WRN  html construction not possible
[1104 15:2

...

7_rotated
Таблицы сохранены в 'output/deepdocetion_results/7_rotated_tables.html'
Текст сохранен в 'output/deepdocetion_results/7_rotated_text.txt'
output/rotated/1_rotated


|          |1/?[00:00<00:00,1331.10it/s]
[1104 15:33.43 @doctectionpipe.py:84]  INF  Processing 1_rotated.png
[1104 15:33.56 @context.py:133]  INF  ImageLayoutService total: 12.9837 sec.
[1104 15:33.56 @context.py:133]  INF  SubImageLayoutService total: 0.0001 sec.
[1104 15:33.56 @context.py:133]  INF  SubImageLayoutService total: 0.0001 sec.
[1104 15:33.56 @context.py:133]  INF  TableSegmentationService total: 0.0001 sec.
[1104 15:33.56 @context.py:133]  INF  TableSegmentationRefinementService total: 0.0001 sec.
[1104 15:34.00 @context.py:133]  INF  TextExtractionService total: 4.4348 sec.
[1104 15:34.00 @context.py:133]  INF  MatchingService total: 0.003 sec.
[1104 15:34.00 @context.py:133]  INF  TextOrderService total: 0.0054 sec.


1_rotated
Таблицы не найдены.
Текст сохранен в 'output/deepdocetion_results/1_rotated_text.txt'
output/rotated/2_rotated


|          |1/?[00:00<00:00,2700.78it/s]
[1104 15:34.01 @doctectionpipe.py:84]  INF  Processing 2_rotated.png
[1104 15:34.14 @context.py:133]  INF  ImageLayoutService total: 13.0108 sec.
[1104 15:34.29 @context.py:133]  INF  SubImageLayoutService total: 15.5017 sec.
[1104 15:34.42 @context.py:133]  INF  SubImageLayoutService total: 12.8985 sec.
[1104 15:34.42 @context.py:133]  INF  TableSegmentationService total: 0.2274 sec.
[1104 15:34.42 @context.py:133]  INF  TableSegmentationRefinementService total: 0.0952 sec.
[1104 15:34.46 @context.py:133]  INF  TextExtractionService total: 4.0666 sec.
[1104 15:34.46 @context.py:133]  INF  MatchingService total: 0.0084 sec.
[1104 15:34.46 @context.py:133]  INF  TextOrderService total: 0.0456 sec.
[1104 15:34.50 @view.py:326]  WRN  html construction not possible
[1104 15:34.50 @view.py:326]  WRN  html construction not possible
[1104 15:34.50 @view.py:326]  WRN  html construction not possible
[1104 15:34.50 @view.py:326]  WRN  html construction no

2_rotated
Таблицы сохранены в 'output/deepdocetion_results/2_rotated_tables.html'
Текст сохранен в 'output/deepdocetion_results/2_rotated_text.txt'


In [52]:
import cv2
import easyocr
import os

# Инициализация EasyOCR
reader = easyocr.Reader(['ru'])

# Директория с изображениями
input_directory = 'output'
output_directory = 'output/easy_ocr_output'
os.makedirs(output_directory, exist_ok=True)  # Создаем выходную директорию, если её нет

# Обработка всех файлов в директории
for filename in os.listdir(input_directory):
    if filename.endswith(('.jpg', '.jpeg', '.png')):  # Проверяем, что файл - изображение
        image_path = os.path.join(input_directory, filename)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Ошибка: изображение по пути '{image_path}' не найдено.")
            continue

        # Распознавание текста на изображении
        results = reader.readtext(image)

        # Сохранение распознанного текста
        recognised_text = [text for (_, text, _) in results]

        # Сохранение в файл
        base_name = os.path.splitext(filename)[0]
        output_file = os.path.join(output_directory, f"{base_name}_recognised.txt")
        with open(output_file, 'w') as f:
            for line in recognised_text:
                f.write(line + '\n')

        print(f"Распознанный текст сохранен в '{output_file}'.")

[1104 15:42.16 @easyocr.py:80]  WRN  Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
[1104 15:42.16 @easyocr.py:251]  WRN  Downloading detection model, please wait. This may take several minutes depending upon your network connection.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

[1104 15:42.25 @easyocr.py:255]  INF  Download complete
[1104 15:42.25 @easyocr.py:176]  WRN  Downloading recognition model, please wait. This may take several minutes depending upon your network connection.


Progress: |██████████████████████████████████████████████████| 100.1% Complete

[1104 15:42.27 @easyocr.py:180]  INF  Download complete.


Распознанный текст сохранен в 'output/easy_ocr_output/1_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/6_rotated_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/6_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/3_rotated_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/5_rotated_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/2_rotated_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/4_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/2_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/8_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/10_rotated_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/9_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/3_recognised.txt'.
Распознанный текст сохранен в 'output/easy_ocr_output/8_rotated_recognised.txt'.
Ра

In [51]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 17.8 MB/s eta 0:00:00
